In [1]:
from helpers import *
import cv2
import numpy as np
import warnings
warnings.filterwarnings("ignore")
%matplotlib qt

In [2]:
def run_sfm(dataset_number):
    K, img_names, init_pair, pixel_threshold = get_dataset_info(dataset_number)
    images = [cv2.cvtColor(cv2.imread(img_name), cv2.COLOR_BGR2GRAY) for img_name in img_names]
    num_images = len(img_names)
    focal_length = K[0][0]
    epipolar_threshold = pixel_threshold / focal_length
    homography_threshold = 3 * pixel_threshold / focal_length
    translation_threshold = 3 * pixel_threshold / focal_length
    sift = cv2.SIFT_create()
    P1 = np.eye(3,4)
    Rs = []
    Rs.append(P1[:3,:3])
    step6_inliers = []
    
    
    for i in range(num_images - 1):
        x1, x2 = image_points(images[i], images[i + 1])
        x1_hom  = np.vstack((x1, np.ones(x1.shape[1])))
        x2_hom  = np.vstack((x2, np.ones(x2.shape[1])))
        x1n = normalize_points(x1_hom, K)
        x2n = normalize_points(x2_hom, K)
        E_best, inliers = estimate_E_robust(x1n, x2n,K, epipolar_threshold)
        step6_inliers.append(inliers)
        x1_inliers = x1n[:, inliers]
        x2_inliers = x2n[:, inliers]
        
        P2,_ = best_P(E_best,K,P1,x1_inliers,x2_inliers)
        R = P2[:, :3]
        
        upgrade_R = R @ Rs[i]
        Rs.append(upgrade_R)


    i1, i2 = init_pair[0] - 1, init_pair[1] - 1
    X0_world, desc_X = reconstruct_initial_3D(i1,i2,images,epipolar_threshold, K, P1, Rs)

    Ts = []
    for i in range(num_images):
        kp, desc = sift.detectAndCompute(images[i], None)
        bf = cv2.BFMatcher()
        matches = bf.knnMatch(desc, desc_X, k=2)
    
        filtered_matches = []
        for m, n in matches:
            if m.distance < 0.80 * n.distance:
                filtered_matches.append(m)
        
        xs = np.array([kp[m.queryIdx].pt for m in filtered_matches]).T
        Xs = X0_world[:, [m.trainIdx for m in filtered_matches]]

        xs_hom = np.vstack((xs, np.ones(xs.shape[1])))
        xsn = normalize_points(xs_hom,K)
        
        P, inliers_idx = estimate_T_robust(xsn, Xs, Rs[i], translation_threshold)
        Ts.append(P[:, -1])


    final_X = []
    Ps = []
    for i in range(len(images)): 
        Ps.append(np.hstack((Rs[i], Ts[i].reshape(-1, 1))))
    
    for i in range(len(images) - 1):
        kp1, des1 = sift.detectAndCompute(images[i], None)
        kp2, des2 = sift.detectAndCompute(images[i+1], None)
        
        bf = cv2.BFMatcher()
        matches = bf.knnMatch(des1, des2, k=2)
    
        filtered_matches = []
        for m, n in matches:
            if m.distance < 0.80 * n.distance:
                filtered_matches.append(m)
        
        x1 = np.array([kp1[m.queryIdx].pt for m in filtered_matches]).T
        x2 = np.array([kp2[m.trainIdx].pt for m in filtered_matches]).T
        
        x1_hom = np.vstack((x1, np.ones(x1.shape[1])))
        x2_hom = np.vstack((x2, np.ones(x2.shape[1])))
    
        x1n = normalize_points(x1_hom, K)
        x2n = normalize_points(x2_hom, K)
     
        P1 = Ps[i]
        P2 = Ps[i+1]
    
        X = triangulate_all_points(P1, P2, x1n[:,step6_inliers[i]], x2n[:,step6_inliers[i]])
    
        X_mean = np.mean(X, axis=1)
        X_dist = np.linalg.norm(X - X_mean[:, np.newaxis], axis=0)
        
        threshold = 2 * np.percentile(X_dist, 90)
        X_inliners = X[:, X_dist <= threshold]
        
    
        final_X.append(X_inliners)

    
    plot_3d_points_and_cameras_with_principal_axes(final_X,Ps)

In [3]:
# Change input number in "run_sfm()" to the dataset you want to run
run_sfm(3)